In [6]:
from langchain.llms import GooglePalm

#Google PaLM LLM Connectivity

api_key = "google-api-key"

llm = GooglePalm(google_api_key=api_key, temperature=0.2)

print(llm("Write a poem for mountain and beaches"))

**Mountains and Beaches**

The mountains and the beaches,
Two sides of a coin,
One so rough and rugged,
The other so smooth and serene.

The mountains, towering high,
Their peaks capped with snow,
A challenge to the climbers,
A feast for the eyes.

The beaches, stretching out wide,
Their sands kissed by the waves,
A place to relax and unwind,
A playground for the children.

The mountains and the beaches,
Two opposites,
Yet both so beautiful,
In their own way.

I love the mountains for their strength,
And the beaches for their beauty.
I am grateful for the chance to experience
Both of these wonders of nature.

When I am feeling stressed,
I go to the mountains,
To breathe in the fresh air
And clear my head.

When I am feeling tired,
I go to the beach,
To relax in the sun
And let the waves wash over me.

The mountains and the beaches,
Two places that I love,
Two places that I will always cherish.


In [8]:
from langchain.utilities import SQLDatabase

#DB Connectivity

db_user = "root"
db_password = "password"
db_host= "localhost"
db_name = "4tees_tshirts"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)
print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [12]:
from langchain_experimental.sql import SQLDatabaseChain

# Asking questions which internally creates and executes a query

db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns1 = db_chain("How many t-shirts do we have left for nike in extra small size and red color")



> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for nike in extra small size and red color
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND size = 'XS' AND color = 'Red'
SQLResult: [(26,)]
Answer:26
> Finished chain.


In [13]:
qns2 = db_chain("SELECT sum(price*stock_quantity) FROM 4tees_tshirts.t_shirts WHERE size = 'S'")



> Entering new SQLDatabaseChain chain...
SELECT sum(price*stock_quantity) FROM 4tees_tshirts.t_shirts WHERE size = 'S'
SQLQuery:SELECT sum(price*stock_quantity) FROM 4tees_tshirts.t_shirts WHERE size = 'S'
SQLResult: [(Decimal('22086'),)]
Answer:22086
> Finished chain.


In [14]:
qns3 = db_chain("""SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """)



> Entering new SQLDatabaseChain chain...
SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('31726.800000'),)]
Answer:31726.8
> Finished chain.


In [48]:
#FEW SHOT LEARNING
few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns1},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': qns2},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': qns3} ,
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : "31850"},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : "221"
     },
    {'Question': "how much sales amount will be generated if we sell all large size t shirts today in nike brand after discounts?",
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Nike' and size="L"
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer' : "3000"
    }
]

In [23]:
from langchain.embeddings import HuggingFaceEmbeddings

# Using HuggingFace Embeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

#Below is an example of creating an embedding for a query
e1 = embeddings.embed_query("How many white color Levi's shirt I have")

In [24]:
e1[:5]

[0.006943077780306339,
 0.04817471280694008,
 0.022458195686340332,
 -0.014069988392293453,
 0.06808184087276459]

In [53]:
to_vectorize = [example.values() for example in few_shots]
to_vectorize

[dict_values(['How many t-shirts do we have left for Nike in XS size and white color?', "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'", 'Result of the SQL query', {'query': 'How many t-shirts do we have left for nike in extra small size and red color', 'result': '26'}]),
 dict_values(['How much is the total price of the inventory for all S-size t-shirts?', "SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'", 'Result of the SQL query', {'query': "SELECT sum(price*stock_quantity) FROM 4tees_tshirts.t_shirts WHERE size = 'S'", 'result': '22086'}]),
 dict_values(['If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?', "SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join 

In [ ]:
from langchain.vectorstores import Chroma

#Creating vectorstore

vectorstore = Chroma.from_texts(to_vectorize, embedding=embeddings, metadatas=few_shots)

In [ ]:
from langchain.prompts import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector(vectorstore=vectorstore,k=2)
example_selector.select_examples({"Question": "How many Adidas T shirts I have left in my store"})

In [64]:
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [67]:
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
        Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
        Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
        Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
        Pay attention to use CURDATE() function to get the current date, if the question involves "today".

        Use the following format:

        Question: Question here
        SQLQuery: Query to run with no pre-amble
        SQLResult: Result of the SQLQuery
        Answer: Final answer here

        No pre-amble.
        """

In [65]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [ ]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables = ["Question", "SQLQuery", "SQLResult", "Answer"],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}"
)

In [ ]:
from langchain.prompts import FewShotPromptTemplate

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"] #These variables are used in prefix and suffix
)
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

new_chain("How many white color Levi's short I have?")